In [10]:
import os
import pandas as pd
import numpy as np
import re

In [4]:
# Imported the dataset
from datasets import load_dataset
from datasets import Dataset

train_set1=load_dataset('ai4privacy/pii-masking-300k', split='train')
val_set=load_dataset('ai4privacy/pii-masking-300k', split='validation')

filepath = os.path.join(os.getcwd(), '..', 'data', 'maker_day_shrieyaa_stella_mini200_df.csv')
train_df2 = pd.read_csv(filepath)
train_set2 = Dataset.from_pandas(train_df2)

In [5]:
# Inspect the dataset
print(type(train_set2))
print(train_set2.shape)
val_set.shape

<class 'datasets.arrow_dataset.Dataset'>
(200, 3)


(47728, 9)

In [6]:
import random
# Select 100 random indices
# random_indices = random.sample(range(len(train_set1)), 10)
random_indices = random.sample(range(len(train_set2)), 10)

# Get the random samples
# random_samples = train_set1.select(random_indices)
random_samples = train_set2.select(random_indices)

print(random_samples)

Dataset({
    features: ['Unnamed: 0', 'unmasked_text', 'masked_text'],
    num_rows: 10
})


In [7]:
import random
# Select 100 random indices
# random_indices = random.sample(range(len(train_set1)), 10)
random_indices = random.sample(range(len(train_set2)), 10)

# Get the random samples
random_samples = train_set2.select(random_indices)

print(random_samples)

Dataset({
    features: ['Unnamed: 0', 'unmasked_text', 'masked_text'],
    num_rows: 10
})


In [8]:
# Load model directly
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('lakshyakh93/deberta_finetuned_pii')
tokenizer.vocab

{'Ġdiabetes': 7704,
 'ĠLiving': 10427,
 'Ġbombs': 10834,
 'ĠAnders': 22786,
 'Ġgays': 34936,
 'Ġspeaker': 5385,
 'Ġstressful': 17322,
 'Dead': 33838,
 'atherine': 38687,
 'opers': 18158,
 'standard': 21993,
 'ĠZhou': 28279,
 'GU': 37133,
 'ĠMiguel': 12450,
 'ĠHL': 38690,
 'ĠEssence': 41845,
 'trop': 42629,
 'ĠPatreon': 44452,
 'Calling': 45424,
 'anga': 9615,
 'ĠSiberia': 36554,
 'Ġprayer': 9621,
 'ĠTan': 7650,
 'Ġbearded': 38048,
 'meaning': 24872,
 'Led': 40794,
 'ĠSplash': 31868,
 'Ġdigit': 16808,
 'Ġobject': 7626,
 'sa': 11146,
 'Ġlev': 23035,
 'ĠTale': 21802,
 'ĠKorean': 2238,
 'Ġtriangular': 42265,
 'Ġmembranes': 42037,
 'Ġthinks': 4265,
 'Ġdepiction': 31388,
 'Ġrm': 45900,
 'ĠWW': 15584,
 'Ġlying': 6480,
 'ĠArri': 14498,
 'Ġadvisors': 18727,
 'ĠCaliforn': 19970,
 'ĠCommercial': 9501,
 'Ġvintage': 12669,
 'Ġconflict': 3050,
 'ĠOL': 19763,
 'Ġscaled': 20819,
 'Ġpasser': 17663,
 'file': 21710,
 'ĠSummer': 6524,
 'Ġpenetrated': 38492,
 'Ġpeace': 1987,
 'police': 24848,
 'Ġmetabol': 

In [13]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import evaluate
import torch

# Define and load the tokenizer and dataset
tokenizer = AutoTokenizer.from_pretrained('lakshyakh93/deberta_finetuned_pii')
train_set1_selected = train_set1.select(range(5))

# Define label mapping
# label_map = {'O': 0, 'TIME': 1, 'DATE': 2, 'LASTNAME1': 3, 'LASTNAME2': 4, 'EMAIL': 5, 'SOCIALNUMBER': 6}
label_map = {'O': 0, 'FIRSTNAME_1': 1, 'LASTNAME_1': 2, 'EMAIL_1': 3, 'PHONENUMBER_1': 4, 'SSN_1': 5, 'JOBTITLE_1': 6,
             'SEX_1': 7, 'BUILDINGNUMBER_1': 8, 'STREET_1': 9, 'DOB_1': 10, 'USERNAME_1': 11, 'AGE_1': 12,
             'PREFIX_1': 13, 'ACCOUNTNUMBER_1': 14}


# Function to tokenize and align labels
def tokenize_and_align_labels_train(example):
    # tokenized_inputs = tokenizer(example['source_text'], truncation=True, padding='max_length', max_length=512)
    tokenized_inputs = tokenizer(example['unmasked_text'], truncation=True, padding='max_length', max_length=512)
    labels = ['O'] * len(tokenized_inputs['input_ids'])
    
    # Adjust indexing to fit the tokenization
    # example['source_text'] = example['source_text'][:512]  # Ensure source_text matches max_length
    example['unmasked_text'] = example['unmasked_text'][:512]  # Ensure source_text matches max_length

    # Extract spans from masked_text
    masked_text = example['masked_text']
    # Find all labels in masked_text using regex
    matches = re.finditer(r'\[([A-Z_]+)\]', masked_text)

    # Create spans and assign labels
    for match in matches:
        label_name = match.group(1)  # Get the label from the match
        start = match.start()
        end = match.end()
        label_id = label_map.get(label_name, 0)

    # for span in example['privacy_mask']:
    #     start, end, label_name = span['start'], span['end'], span['label']
    #     label_id = label_map.get(label_name, 0)
        
        # span_tokens = tokenizer(example['source_text'][start:end], add_special_tokens=False).tokens()
        span_tokens = tokenizer(example['unmasked_text'][start:end], add_special_tokens=False).tokens()

        # Properly create and align labels (assuming BERT tokenizer)
        for i, token in enumerate(tokenized_inputs.tokens()):
            if token in span_tokens:
                labels[i] = label_id

    numeric_labels = [label_map.get(label, -100) for label in labels]
    numeric_labels += [-100] * (len(tokenized_inputs['input_ids']) - len(numeric_labels))
    
    tokenized_inputs['labels'] = numeric_labels
    return tokenized_inputs


In [14]:
# Function to tokenize and align labels
def tokenize_and_align_labels_val(example):
    # tokenized_inputs = tokenizer(example['source_text'], truncation=True, padding='max_length', max_length=512)
    tokenized_inputs = tokenizer(example['unmasked_text'], truncation=True, padding='max_length', max_length=512)
    labels = ['O'] * len(tokenized_inputs['input_ids'])
    
    # Adjust indexing to fit the tokenization
    # example['source_text'] = example['source_text'][:512]  # Ensure source_text matches max_length
    example['unmasked_text'] = example['unmasked_text'][:512]  # Ensure source_text matches max_length

    # Create spans and assign labels
    for span in example['privacy_mask']:
        start, end, label_name = span['start'], span['end'], span['label']
        label_id = label_map.get(label_name, 0)
        
        # span_tokens = tokenizer(example['source_text'][start:end], add_special_tokens=False).tokens()
        span_tokens = tokenizer(example['unmasked_text'][start:end], add_special_tokens=False).tokens()

        # Properly create and align labels (assuming BERT tokenizer)
        for i, token in enumerate(tokenized_inputs.tokens()):
            if token in span_tokens:
                labels[i] = label_id

    numeric_labels = [label_map.get(label, -100) for label in labels]
    numeric_labels += [-100] * (len(tokenized_inputs['input_ids']) - len(numeric_labels))
    
    tokenized_inputs['labels'] = numeric_labels
    return tokenized_inputs

In [15]:
# Apply function to dataset
# encoded_dataset = train_set1_selected.map(tokenize_and_align_labels, batched=False)
encoded_dataset = train_set2.map(tokenize_and_align_labels_train, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(encoded_dataset['input_ids'])
attention_mask = torch.tensor(encoded_dataset['attention_mask'])
labels = torch.tensor(encoded_dataset['labels'])

# Load model
model = AutoModelForTokenClassification.from_pretrained('lakshyakh93/deberta_finetuned_pii')

# Define metric computation
metric = evaluate.load('accuracy')  # You may want to load relevant metrics for token classification

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=2)  # Note: axis=2 for token classification
    true_labels = labels != -100  # Masking out unnecessary labels
    
    # Flatten predictions and true_labels to compute accuracy
    flattened_predictions = predictions[true_labels]
    flattened_labels = labels[true_labels]
    
    results = metric.compute(references=flattened_labels, predictions=flattened_predictions)
    return results


# Tokenize validation set
small_val = val_set.shuffle(seed=123).select(range(5))
encoded_small_val = small_val.map(tokenize_and_align_labels_val, batched=False)

# Training arguments
train_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-2,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=10,
    eval_steps=10,
    gradient_accumulation_steps=2,  # Adjust if needed
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_small_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

KeyError: 'unmasked_text'

In [ ]:
# Run training
trainer.train()

 33%|███▎      | 1/3 [00:56<01:24, 42.44s/it]

{'eval_loss': 11.556055068969727, 'eval_accuracy': 0.0, 'eval_runtime': 13.9542, 'eval_samples_per_second': 0.358, 'eval_steps_per_second': 0.072, 'epoch': 1.0}


In [ ]:
trainer.evaluate()

100%|██████████| 2/2 [00:04<00:00,  2.25s/it]


{'eval_loss': 0.0,
 'eval_accuracy': 1.0,
 'eval_runtime': 22.0797,
 'eval_samples_per_second': 0.453,
 'eval_steps_per_second': 0.091,
 'epoch': 3.0}